## initial code for our Hand Gesture Recognition

In [72]:
# ML models imports
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from skimage.feature import hog, local_binary_pattern

from skimage.filters import try_all_threshold

# destribution models imports
from scipy.stats import randint, uniform

# Data manipulation imports
import numpy as np
import pandas as pd
import tqdm as tqdm
import joblib

# visualisation models imports
import matplotlib.pyplot as plt

# image processing imports
import skimage.io as io
import cv2
from skimage.transform import resize

# dealing with files
import os

# visual dataset (to test randomized gridsearch not needed for now)
from sklearn.datasets import make_hastie_10_2  # to test our models

# from utils import prepareData, LoadData, FeatureExtraction, preprocess
import csv

In [84]:
def segment(image):
    blured_image = cv2.GaussianBlur(image, (7, 7), 0)
    
    # Extract the Cr channel
    ycbcr_image = cv2.cvtColor(blured_image, cv2.COLOR_BGR2YCrCb)
    cr_channel = ycbcr_image[:,:,1]

    # Apply thresholding to obtain a binary image
    _, binary_img = cv2.threshold(cr_channel,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Define the structuring element for the closing operation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    # Perform the closing operation
    closed_img = cv2.dilate(binary_img, kernel, iterations=1)

    # Find the contours in the binary image
    contours, hierarchy = cv2.findContours(closed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    segmented_image = closed_img.copy()
    cv2.drawContours(segmented_image, contours, -1, 255, -1)
    
    # preprocessed = cv2.bitwise_and(image, image, mask=segmented_image)
    # preprocessed = cv2.cvtColor(preprocessed, cv2.COLOR_BGR2GRAY)

    return segmented_image, contours

In [47]:
from scipy.signal import find_peaks

def count_peaks(image, contours):
    M = cv2.moments(image)
    
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    centroid = np.array([cX, cY])

    # Find the contour with the maximum area
    max_contour = max(contours, key=cv2.contourArea)
    max_contour = np.squeeze(max_contour)
    # Find peaks in the x-coordinate of contour points
    x = max_contour[:, 0]
    
    m, n = image.shape
    if(centroid[0] < n/2): # the hand is pointing to right
        peak_indices, _ = find_peaks(x, distance=200)
    else:  # the hand is pointing to left
        peak_indices, _ = find_peaks(-x, distance=200)
    
    peaks = max_contour[peak_indices]
    
    if len(peak_indices) == 0: return 0

    distance = np.linalg.norm(peaks - centroid, axis=1)
    max_peak_distance = np.max(distance)

    significant_peaks = peaks[distance >= 0.75*max_peak_distance]

    # Draw circles around peak points on the original image
    # for peak in peaks:
    #     cv2.circle(image, peak, 50, (0, 255, 0), -1)
    
    return(len(significant_peaks))

def ratio(contours):
    # Find the contour with the maximum area
    max_contour = max(contours, key=cv2.contourArea)
    max_contour = np.squeeze(max_contour)

    area = cv2.contourArea(max_contour)
    x, y, w, h = cv2.boundingRect(max_contour)
    area_bounding_rectangle = w*h
    ratio = area/area_bounding_rectangle
    return ratio

In [63]:
def FeatureExtraction(image):
        
    # Extract the hog features
    # block_norm uses L2 norm with hysterisis for reducing effect of illuminacity
    # transform_sqrt for applying gamma correction
    preprocessed_image, contours = segment(image)
    
    resized_image = resize(preprocessed_image,(64,128))

    # hog_features = hog(resized_image, block_norm='L2-Hys', feature_vector=True, transform_sqrt=True, pixels_per_cell=(12, 12), cells_per_block=(2, 2))
    contour_to_ROI = ratio(contours)
    peak_num = count_peaks(preprocessed_image, contours)
    
    # features = np.append(hog_features, (contour_to_ROI, peak_num))
    features = np.array([contour_to_ROI, peak_num])

    return features

In [85]:
def testOurModel(path):
    '''
       this is a utility function used to load the test data
    '''

    Features = []
    datadir = path
    # loop over gender
    counter=1
    for img in sorted(os.listdir(datadir)):
        # ignoring anything except images
        if((img.split('.')[-1]).lower() not in ['jpg', 'png', 'jpeg']):
            continue
        if counter>54:
            ourPath= datadir + "/" + '(' + str(counter)+").jpeg"
        else:
            ourPath= datadir + "/" + '(' + str(counter)+").jpg"   
        print(ourPath)
        # loading our images
        img_array = io.imread(ourPath)
        counter=counter+1
        # append extracted features to Featurees list
        Features.append(FeatureExtraction(img_array))

    # here i have features need to pass them to my model

    svm = joblib.load('kiro_model.pkl')

    y_pred = svm.predict(pd.DataFrame(Features))

    return y_pred


def readFinalResults(resultFilePath):
    ''' 
        this is a utility function used to read the 
        file which contains the actual results for 
        the test data. 
        input:
            resultFilePath: the path of the file which contains the actual results for the test data.
        output:
            results: the actual results for the test data.
    '''
    f = open(resultFilePath, 'r')
    results = f.read()
    f.close()
    return results

In [86]:
def LoadData():
    Features=[]
    labels=[]

    i = 0

    
    for gender in ["men","Women"]:
        datadir = r"Dataset\{}".format(gender)
        # loop over gender
        for hand in os.listdir(datadir): 
            # loop over each class [0,1,2,3,4,5]
            for img in os.listdir(datadir+ "/" +str(hand)):
                # ignoring anything except images
                if((img.split('.')[-1]).lower() not in ['jpg','png','jpeg']):
                    continue

                # loading our images
                img_array=io.imread(datadir + "/" + str(hand) + "/" + img )  # approx 2500 * 4000

                # append extracted features to Featurees list   
                Feature = FeatureExtraction(img_array)        
                Features.append(Feature) 

                # append class of image.
                labels.append(hand)  

                print(f'image Number: {i}')
                i+=1
                # # print(f'saving block : {i//100}')
                # # np.save(f'Features/{i}.npy', Feature)
                # # Features = []
                # # labels = []
                # with open("Features.csv", 'a') as csvfile:
                #     csvwriter = csv.writer(csvfile)
                #     csvwriter.writerow(Feature)

    return np.asarray(Features),np.asarray(labels)

In [87]:
Features, labels = LoadData() 

image Number: 0
image Number: 1
image Number: 2
image Number: 3
image Number: 4
image Number: 5
image Number: 6
image Number: 7
image Number: 8
image Number: 9
image Number: 10
image Number: 11
image Number: 12
image Number: 13
image Number: 14
image Number: 15
image Number: 16
image Number: 17
image Number: 18
image Number: 19
image Number: 20
image Number: 21
image Number: 22
image Number: 23
image Number: 24
image Number: 25
image Number: 26
image Number: 27
image Number: 28
image Number: 29
image Number: 30
image Number: 31
image Number: 32
image Number: 33
image Number: 34
image Number: 35
image Number: 36
image Number: 37
image Number: 38
image Number: 39
image Number: 40
image Number: 41
image Number: 42
image Number: 43
image Number: 44
image Number: 45
image Number: 46
image Number: 47
image Number: 48
image Number: 49
image Number: 50
image Number: 51
image Number: 52
image Number: 53
image Number: 54
image Number: 55
image Number: 56
image Number: 57
image Number: 58


In [88]:
labels = readFinalResults('results_set_2.txt')
y_pred = testOurModel('testData')

testData/(1).jpg
testData/(2).jpg
testData/(3).jpg
testData/(4).jpg
testData/(5).jpg
testData/(6).jpg
testData/(7).jpg
testData/(8).jpg
testData/(9).jpg
testData/(10).jpg
testData/(11).jpg
testData/(12).jpg
testData/(13).jpg
testData/(14).jpg
testData/(15).jpg
testData/(16).jpg
testData/(17).jpg
testData/(18).jpg
testData/(19).jpg
testData/(20).jpg
testData/(21).jpg
testData/(22).jpg
testData/(23).jpg
testData/(24).jpg
testData/(25).jpg
testData/(26).jpg
testData/(27).jpg
testData/(28).jpg
testData/(29).jpg
testData/(30).jpg
testData/(31).jpg
testData/(32).jpg
testData/(33).jpg
testData/(34).jpg
testData/(35).jpg
testData/(36).jpg
testData/(37).jpg
testData/(38).jpg
testData/(39).jpg
testData/(40).jpg
testData/(41).jpg
testData/(42).jpg
testData/(43).jpg
testData/(44).jpg
testData/(45).jpg
testData/(46).jpg
testData/(47).jpg
testData/(48).jpg
testData/(49).jpg
testData/(50).jpg
testData/(51).jpg
testData/(52).jpg
testData/(53).jpg
testData/(54).jpg
testData/(55).jpeg
testData/(56).jpeg

In [90]:
print(1-(np.sum(labels[54:] != y_pred[54:]))/len(labels[54:]))

0.6534090909090908


* # Selecting the best model

- ### Define hyperparameter grids for each model

In [26]:
param_distributions = {
    'RandomForestClassifier': {
        'n_estimators': randint(50, 500),
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'GradientBoostingClassifier': {
        'learning_rate': uniform(0.01, 0.2),
        'n_estimators': randint(50, 500),
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'SVC': {
        'C': uniform(0.01, 10),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'degree': randint(2, 5),
        'gamma': ['scale', 'auto'] + list(np.arange(0.1, 1, 0.1))
    },
    'LogisticRegression': {
        'C': uniform(0.01, 10),
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'max_iter': randint(50, 500)
    },
    'DecisionTreeClassifier': {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': randint(2, 20),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 5),
        'max_features': ['sqrt', 'log2']
    },
    'KNeighborsClassifier': {
        'n_neighbors': randint(3, 30),
        'weights': ['uniform', 'distance'],
        'algorithm': ['ball_tree', 'kd_tree', 'brute'],
        'leaf_size': randint(10, 100)
    },
    'GaussianNB': {
        'var_smoothing': uniform(1e-09, 1e-07)
    },
    'MLPClassifier': {
        'hidden_layer_sizes': [(50, 50), (100,), (100, 50)],
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'alpha': uniform(0.0001, 0.01),
        'max_iter': randint(100, 1000)
    }
}

- ### Create a list of models to train (as example)

In [27]:
models = [
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    SVC(),
    KNeighborsClassifier(),
    GaussianNB()
]

- ### Loop over the models and fit  

In [28]:
# load our dummy data to test the randomizedSearche function
x,y = Features,labels
df = pd.DataFrame(x)
df['Y'] = y

train, test = train_test_split(df, test_size=0.2) # this function shuffles the data points, and splits the data into
                                                  # 80% training set and 20% test set (indicated by test_size=0.2)
X_train, Y_train = train.iloc[:, :-1], train.iloc[:, -1]
X_test, Y_test = test.iloc[:, :-1], test.iloc[:, -1]

In [30]:
for i, model in enumerate(models):
    print(f'Training Model {i+1}/{len(models)}: {str(model)[:-2]}')
    # Define randomized grid search
    random_search = RandomizedSearchCV(model, param_distributions[str(model)[:-2]], n_iter=10,cv=5, n_jobs=-1) # n_jobs means number of jobs to run in parallel. None means 1,
                                                                                                                # -1 means using all processors 😈.
    # Fit the randomized grid search to the data
    random_search.fit(X_train, Y_train)
    print(f'Best score: {random_search.best_score_:.3f}')
    print(f'Best parameters: {random_search.best_params_}\n')

Training Model 1/5: RandomForestClassifier
Best score: 0.400
Best parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 130}

Training Model 2/5: GradientBoostingClassifier
Best score: 0.424
Best parameters: {'learning_rate': 0.03703344575971452, 'max_depth': 8, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 68}

Training Model 3/5: SVC
Best score: 0.451
Best parameters: {'C': 8.948624388292055, 'degree': 4, 'gamma': 0.30000000000000004, 'kernel': 'poly'}

Training Model 4/5: KNeighborsClassifier
Best score: 0.360
Best parameters: {'algorithm': 'kd_tree', 'leaf_size': 31, 'n_neighbors': 9, 'weights': 'uniform'}

Training Model 5/5: GaussianNB
Best score: 0.384
Best parameters: {'var_smoothing': 8.281483955122005e-08}

